In [2]:
#!pip install python-twitter

In [26]:
import pandas as pd
import twitter, json,re,sqlalchemy
from sqlalchemy.sql import select
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table,Column,Integer,String,TEXT,create_engine,UniqueConstraint

In [6]:
Base = declarative_base()

In [18]:
class Tweet_message(Base):
  '''
  Table schema for Message
  '''
  __tablename__ = 'tweet_message'
  __table_args__ = ( UniqueConstraint('keyword'),)

  message_id  = Column(Integer, primary_key=True, nullable=False)
  keyword     = Column(String(50), nullable=False)
  message_out = Column(TEXT())

In [19]:
engine=create_engine('sqlite:///:memory:')

In [20]:
Base.metadata.create_all(engine)

In [21]:
Session=sessionmaker(bind=engine)

In [22]:
session=Session()

In [27]:
query=session.query(Tweet_message)

In [28]:
pd.read_sql(query.statement, session.bind)

,message_id,keyword,message_out


In [29]:
def insert_dataframe_row(session, table, df):
    df=df.fillna('')
    df_dict=df.to_dict()
    try:
        df_dict={ key:value for key, value in df_dict.items() if value} # filter any empty value
        obj=table(**df_dict)
        session.add(obj)
        session.flush()
    except Exception as e:
        print("Failed to insert row, error: {}".format(e))
        session.rollback()

In [31]:
message_data=[{'keyword':'sample_submission',
               'message_out':\
               'Please follow these steps: \
                1. Download, complete and send us this form to receive a quote https://www.imperial.ac.uk/terminalfour/SiteManager?ctfn=download&fnno=60&ceid=306973353 \
                2. Check per sample recommended read depth https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/faq/coverage-and-read-recommendations/ \
                3. Check assay specific submission guide https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/guidelines/ \
                4. Deliver samples to facility',
              },
              {'keyword':'assay_samples',
               'message_out':'Check the assay specific sample submission guide https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/guidelines/',
              },
              {'keyword':'sequencing_cost',
               'message_out':'You will require an Imperial College\'s login for accessing the sequencing cost information https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/services/pricing/',
              },
              {'keyword':'contact_us',
               'message_out':'Our contact infomation can be found here https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/contact/',
              },
             ]

In [33]:
message_data=pd.DataFrame(message_data)

In [34]:
message_data.apply(lambda x: insert_dataframe_row(session,Tweet_message, x), axis=1);

0    None
1    None
2    None
3    None
dtype: object

In [35]:
session.commit()

In [40]:
pd.read_sql(query.statement, session.bind)

,message_id,keyword,message_out
0,1,sample_submission,Please follow these steps: ...
1,2,assay_samples,Check the assay specific sample submission gui...
2,3,sequencing_cost,You will require an Imperial College's login f...
3,4,contact_us,Our contact infomation can be found here https...


In [41]:
token_file='twep_token.json'

In [42]:
with open(token_file,'r') as json_data:
    twep_token=json.load(json_data)

In [43]:
api = twitter.Api(twep_token['consumer_key'],
                  twep_token['consumer_secret'],
                  twep_token['access_token_key'],
                  twep_token['access_token_secret'])

In [46]:
quick_reply_data={"type": "options",
                  "options": 
                  [
                   {
                     "label": "sample_submission",
                     "metadata": "external_id_1"
                   },
                   {
                     "label": "assay_samples",
                     "metadata": "external_id_2"
                   },
                   {
                     "label": "sequencing_cost",
                     "metadata": "external_id_3"
                   },
                   {
                     "label": "contact_us",
                     "metadata": "external_id_4"
                   }
                  ]
                 }

In [47]:
welcome_data={
  "welcome_message" : {
    "message_data": {
      "text": "Welcome to IGF bot chat, please select an option from below.",
      "quick_reply": quick_reply_data,
    }
  }
}

In [49]:
post_data={'event': {
             "type": "message_create",
             "message_create": {
               "target": {
                 "recipient_id": None,
               },
               "message_data": {  
                 "text": None,
                 "quick_reply": quick_reply_data,
               },
              },
            }
          }

In [51]:
url='{0}/direct_messages/events/new.json'.format(api.base_url)

In [54]:
def get_message_outpur(keyword,session=session):
    try:
        message=session.query(Tweet_message).filter(Tweet_message.keyword==keyword).one()
        return message.message_out
    except:
        raise

In [57]:
strm=api.GetUserStream()

In [59]:
for strm_line in strm:
    post_data["event"]["message_create"]["target"]["recipient_id"]=None
    post_data["event"]["message_create"]["message_data"]["text"]=None
    for strm_key, strm_value in strm_line.items():
        if strm_key=='direct_message':
            sender_name=strm_value['sender_screen_name']
            if not sender_name=='dailybot001':
                msg=get_message_outpur(keyword=strm_value['text'])
                sender_id=strm_value['sender_id']
                post_data["event"]["message_create"]["target"]["recipient_id"]=sender_id
                post_data["event"]["message_create"]["message_data"]["text"]=msg
                data=json.dumps(post_data)
                api._RequestUrl(url, 'POST', data=data)